<a href="https://colab.research.google.com/github/JackBlake-zkq/robust-edge-inference/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[FA Code](https://colab.research.google.com/github/JackBlake-zkq/robust-edge-inference/blob/main/train.ipynb)

[Distillation Code](https://github.com/ZuchniakK/MTKD)